# Bia-bob
This notebook is used to generate a python-file that serves as interoperability connector to [bia-bob](https://github.com/haesleinhuepf/bia-bob). You need an OpenAI account to run this notebook. Note: It uses chatGPT in a loop and executing this notebook costs some cents.

In [1]:
import stackview
import openai

In [2]:
def prompt(message:str, model="gpt-3.5-turbo"):
    """A prompt helper function that sends a message to openAI
    and returns only the text response.
    """
    client = openai.OpenAI()
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": message}]
    )
    return response.choices[0].message.content

In [3]:
all = {}
for n in dir(stackview):
    if not n.startswith('_'):
        all[n] = getattr(stackview, n)

all.keys()

dict_keys(['annotate', 'assist', 'create_colormap', 'crop', 'curtain', 'insight', 'interact', 'jupyter_displayable_output', 'merge_rgb', 'nop', 'orthogonal', 'picker', 'side_by_side', 'slice', 'switch'])

In [4]:
all.pop('jupyter_displayable_output')
all.pop('create_colormap')
all.pop('merge_rgb')
all.pop('nop')
all.pop('assist')

all

{'annotate': <function stackview._annotate.annotate(image, labels, slice_number: int = None, alpha: float = 0.5, axis: int = 0, continuous_update: bool = True, slider_text: str = 'Slice', zoom_factor: float = 1.0, zoom_spline_order: int = 0, colormap: str = None, display_min: float = None, display_max: float = None)>,
 'crop': <function stackview._crop.crop(image, slice_number: int = None, axis: int = 0, continuous_update: bool = True, slider_text: str = 'Slice', axis_names=None, zoom_factor: float = 1.0, zoom_spline_order: int = 0, colormap: str = None, display_min: float = None, display_max: float = None)>,
 'curtain': <function stackview._curtain.curtain(image, image_curtain, slice_number: int = None, axis: int = 0, display_width: int = None, display_height: int = None, continuous_update: bool = True, alpha: float = 1, zoom_factor: float = 1.0, zoom_spline_order: int = 0, colormap: str = None, display_min: float = None, display_max: float = None, curtain_colormap: str = None, curtai

In [5]:
def get_function_signature(func):
    import inspect
    
    function_object = func
    signature_object = inspect.signature(function_object)
    signature_string = "stackview." + func.__name__ + str(signature_object)
    return signature_string

all_descriptions = []
for name, func in all.items():
    
    description = prompt(f"""Shorten the following docstring into a single bullet point:

    ```
    {func.__doc__}
    ```
    
    Shorten it to a single bullet point explaining what the function does without mentioning the function name and without phrases such as `use this function`. Do not mention and explain all parameters. Keep it short.

    Examples:

    * compute the absolute value of every individual pixel in an image
    cle.absolute(source: ndarray, destination: ndarray = None) -> ndarray

    * determine the absolute difference pixel by pixel between two images
    cle.absolute_difference(source1: ndarray, source2: ndarray, destination: ndarray = None) -> ndarray
    
    * add a scalar value to all pixels of an image
    cle.add_image_and_scalar(source: ndarray, destination: ndarray = None, scalar: float = 1) -> ndarray
    
    """).replace("If you want to ", "* ")
    description = description + "\n" + get_function_signature(func) + "\n\n"
    print(description)
    
    all_descriptions.append(description)

print(all_descriptions[:1000])

Displays an image with a slider and label showing mouse position and intensity.
stackview.annotate(image, labels, slice_number: int = None, alpha: float = 0.5, axis: int = 0, continuous_update: bool = True, slider_text: str = 'Slice', zoom_factor: float = 1.0, zoom_spline_order: int = 0, colormap: str = None, display_min: float = None, display_max: float = None)


Allows cropping an image along all axes.
stackview.crop(image, slice_number: int = None, axis: int = 0, continuous_update: bool = True, slider_text: str = 'Slice', axis_names=None, zoom_factor: float = 1.0, zoom_spline_order: int = 0, colormap: str = None, display_min: float = None, display_max: float = None)


- Display two images and use a slider to switch between them.
stackview.curtain(image, image_curtain, slice_number: int = None, axis: int = 0, display_width: int = None, display_height: int = None, continuous_update: bool = True, alpha: float = 1, zoom_factor: float = 1.0, zoom_spline_order: int = 0, colormap: str = No

In [6]:
descriptions = []
for bullet_point in all_descriptions:
    if bullet_point[0:2] in ['- ', '* ']:
        bullet_point = bullet_point[2:]

    descriptions.append("* " + bullet_point)


In [7]:
text = "".join(descriptions).replace('"', "'")
text = "    " + text.replace("\n", "\n    ")
text = f'''
def list_bia_bob_plugins():
    """List of function hints for bia_bob"""
    return """{text}"""
'''

with open("../stackview/_bia_bob_plugins.py", 'w') as file:
    file.write(text)